In [ ]:
import pandas as pd
import numpy as np
import os
import numpy
import matplotlib.pyplot as plt
import SimpleITK
import itertools
import sys
import torch
from torchvision import transforms
from PIL import Image
from matplotlib import cm
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from torchvision import models

from pathlib import Path

SOURCE_PATH = Path(os.getcwd()) / 'src'

if SOURCE_PATH not in sys.path:
    sys.path.append(SOURCE_PATH)

from src.extraction import (
    extract_images_in_survival_order,
    export_images_list_jpeg,
)


from src.survival.survival_custom import (
    get_survival_dataloaders,
    SurvivalNet)

from src.utils import(
    LOGS_FILE_PATH,
    TYPE_NAMES
)

%load_ext autoreload
%autoreload 2

In [ ]:
data_path = Path(os.getcwd()) / 'data' / 'HGG'
survival = pd.read_csv(data_path.parent /'survival_data.csv')

In [ ]:
dir_ids = survival['BraTS19ID']

In [ ]:
t2, t1ce, t1, flair, seg = extract_images_in_survival_order(data_path, dir_ids)
images = [t2, t1ce, t1, flair, seg]

In [ ]:
for i in range(len(TYPE_NAMES)):
    survival[TYPE_NAMES[i]] = images[i] 

In [ ]:
modelname = 'flair_totalpipe_nighttrain.pt'
model = torch.load(Path(os.getcwd()) / 'models' / modelname)
model.eval()

In [ ]:
flair_segout = []

for i in range(len(survival)):
    input_tensor = torch.tensor(survival['flair'][i]).expand(3, -1, -1).type(torch.ShortTensor).float().unsqueeze(0)
    with torch.no_grad():
        output = model(input_tensor)['out'][0][0]
    flair_segout.append(np.array(output))

survival['flair_seg'] = flair_segout

In [ ]:
survival = survival.dropna().reset_index(drop=True)
survival['Survival'] = survival['Survival'].apply(lambda x: str(x))

In [ ]:
to_drop = []
for i in range(len(survival)):
    if 'ALIVE' in survival['Survival'].loc[i]:
        to_drop.append(i)
        
        
survival.drop(to_drop, inplace=True)
print(len(survival))

In [ ]:
survival['Survival'] = survival['Survival'].apply(lambda x: int(x))

In [ ]:
modelname = "custom_made_survivalnet_final.pt"
net = torch.load(Path(os.getcwd()) / "models" / modelname)
net.eval()

In [ ]:
dataloaders = get_survival_dataloaders(survival['flair_seg'].values, survival['Survival'].values)

In [ ]:
print('prediction', net(dataloaders['Test'].dataset[0]['image'].unsqueeze(0).float()))
print('target', dataloaders['Test'].dataset[0]['target'])